In [1]:
import sklearn
import numpy as np
import pandas as pd
import pandas_profiling as pp
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.model_selection import cross_val_score
matplotlib.use('TkAgg')
%matplotlib inline

In [2]:
DATA_PATH = './data/'
df_train_temp = pd.read_csv(DATA_PATH+'train.csv')
df_train = df_train_temp.iloc[:,:-1]
df_test = pd.read_csv(DATA_PATH+'test.csv')
df = pd.concat([df_train,df_test], axis=0)
#df.to_csv("temp.csv")

In [3]:
b_mode = ['MSZoning','Functional','KitchenQual','Exterior2nd','Exterior1st','SaleType']
n_0 = ['BsmtFullBath','BsmtHalfBath','BsmtUnfSF','BsmtUnfSF','BsmtFinSF1','BsmtFinSF2']
n_mode = ['GarageCars']
n_mean = ['GarageArea','TotalBsmtSF']

In [4]:
# 标签众数处理
for i in range(len(b_mode)):
    mode = df[b_mode[i]].describe()['top']
    df[b_mode[i]] = df[b_mode[i]].fillna(value=mode)

# 数值0处理
for i in range(len(n_0)):
    df[n_0[i]] = df[n_0[i]].fillna(value=0.0)

# 数值众数处理
for i in range(len(n_mode)):
    mode = df_test[n_mode[i]].var()
    df[n_mode[i]] = df[n_mode[i]].fillna(value=mode)
    
# 数值平均数处理
for i in range(len(n_mean)):
    mode = df[n_mean[i]].mean()
    df[n_mean[i]] = df[n_mean[i]].fillna(value=mode)

In [5]:
df['MasVnrType'] = df['MasVnrType'].fillna(value='None')
df['MasVnrArea'] = df['MasVnrArea'].fillna(value=0)

begin_year =  df['MasVnrArea'].min()
df['GarageYrBlt'] = df['GarageYrBlt'].fillna(value=0)

mode = df['Electrical'].describe()['top']
df['Electrical'] = df['Electrical'].fillna(value=mode)
# 数字型不需要处理
no_need = ['LotArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF',
           '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
           'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
           'TotRmsAbvGrd', 'Fireplaces', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
           'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
           'YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'MoSold', 'YrSold', 'MasVnrArea']

In [6]:
# 缺失值未处理  第一个KNN 第二个None 第三个0 第四个众数
unsolved_list =['LotFrontage', 'MasVnrType', 'MasVnrArea', 'Electrical']

In [7]:
def onehot(df_name,f_list,k):
    new_feature = pd.get_dummies(df_name[f_list[0]].astype(str))
    for i in range(1,k):
        add_feature = pd.get_dummies(df_name[f_list[i]].astype(str))
        new_feature = pd.concat([new_feature,add_feature], axis=1)
    return new_feature


feature_list = ['MSSubClass', 'MSZoning', 'Street', 'LotShape',
                'LandContour', 'Utilities','LotConfig', 'LandSlope',
                'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
                'OverallQual', 'OverallCond', 'RoofStyle', 'RoofMatl', 'Exterior1st',
                'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating',
                'HeatingQC','CentralAir', 'KitchenQual','Functional',
                'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageCars','GarageQual',
                'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
                'MiscVal', 'SaleType', 'SaleCondition', 'Alley', 'MasVnrType',
                'BsmtQual', 'BsmtCond', 'BsmtExposure','BsmtFinType1','BsmtFinType2', 'Electrical']

new_df = onehot(df,feature_list,len(feature_list))

min_dict = {}
max_dict = {}

for i in range(len(no_need)):  
    num_col = df[no_need[i]]
    min_dict[no_need[i]] = num_col.min()
    max_dict[no_need[i]] = num_col.max()
    
    add_df = (num_col - min_dict[no_need[i]]) / (max_dict[no_need[i]] - min_dict[no_need[i]])
    new_df = pd.concat([new_df,add_df], axis=1)
    
temp_df = pd.concat([new_df,df['LotFrontage']], axis=1)
imputer = KNNImputer(n_neighbors=10)
knn_array = imputer.fit_transform(temp_df)[:,-1].reshape(2919,1)
knn_df = pd.DataFrame(knn_array)[0]

min_dict['LotFrontage'] = knn_df.min()
max_dict['LotFrontage'] = knn_df.max()
add_df = (knn_df - min_dict['LotFrontage']) / (max_dict['LotFrontage'] - min_dict['LotFrontage'])
new_df['LotFrontage'] = add_df

In [8]:
new_df

,120,150,160,180,190,20,30,40,45,50,...,3SsnPorch,ScreenPorch,PoolArea,YearBuilt,YearRemodAdd,GarageYrBlt,MoSold,YrSold,MasVnrArea,LotFrontage
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.949275,0.883333,0.907567,0.090909,0.50,0.12250,0.150685
1,0,0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.753623,0.433333,0.895333,0.363636,0.25,0.00000,0.202055
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.934783,0.866667,0.906661,0.727273,0.50,0.10125,0.160959
3,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.311594,0.333333,0.905301,0.090909,0.00,0.00000,0.133562
4,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.927536,0.833333,0.906208,1.000000,0.50,0.21875,0.215753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,0,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.710145,0.333333,0.000000,0.454545,0.00,0.00000,0.140411
1455,0,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.710145,0.333333,0.892614,0.272727,0.00,0.00000,0.140411
1456,0,0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.637681,0.766667,0.888083,0.727273,0.00,0.00000,0.219178
1457,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.869565,0.700000,0.000000,0.545455,0.00,0.00000,0.154110


In [9]:
# 查看特征中 Nan 百分比
def Nan_count(df):
    NA = df.isnull().mean()
    nan = 100*NA[NA>0].sort_values(ascending=False)
    return nan
# 可以看到 其中 GarageYrBlt 到 PoolQC 确实缺失占比高于17%
NAN = Nan_count(new_df)

In [10]:
NAN

Series([], dtype: float64)

In [11]:
train_df = new_df.iloc[:1460,:]
test_df = new_df.iloc[1460:,:]
y_df = df_train_temp.iloc[:,-1]

train_df.to_csv('train.csv')
test_df.to_csv('test.csv')
y_df.to_csv('y.csv')